# Import comet_ml
- need to be on the top, otherwise error
- comet_ml is a library that helps us to track the experiement hyperparameters and metrics

In [41]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/26edc976238642438c52c93fcd48b388



In [42]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

# Import libraries

In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

In [44]:
# tensorflow 2.7 
import tensorflow as tf 
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential

In [45]:
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Import data and pre-preocess data

In [46]:
df_train_features = pd.read_csv('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

df_valid_features = pd.read_csv('valid_features.csv')
df_valid_labels = pd.read_csv('valid_labels.csv')

df_test_features = pd.read_csv('test_features.csv')

In [47]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

In [48]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)
num_test = scaler.fit_transform(num_test_data)

In [49]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

In [50]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [51]:
x_train = df_train_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_train = df_train_labels['genre'].to_numpy()

x_test = df_valid_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_test = df_valid_labels['genre'].to_numpy()

In [52]:
x_train = np.array(x_train)
x_test = np.array(x_test)

# Adjust weights and define early stop mechanism

In [53]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [54]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)

# Import ktrain for text classification model

In [55]:
import ktrain
from ktrain import text

# Text classification model

In [56]:
trn2, val2, preproc2 = text.texts_from_array(
    x_train=x_train, y_train=y_train,
    x_test=x_test, y_test=y_test,
    ngram_range=1, 
    maxlen=150,
    preprocess_mode='standard'
)

language: en
Word Counts: 4888
Nrows: 7678
7678 train sequences
train sequence lengths:
	mean : 80
	95percentile : 139
	99percentile : 193
x_train shape: (7678,150)
y_train shape: (7678, 8)
Is Multi-Label? False
450 test sequences
test sequence lengths:
	mean : 74
	95percentile : 142
	99percentile : 180
x_test shape: (450,150)
y_test shape: (450, 8)
task: text classification


In [57]:
model2 = text.text_classifier('nbsvm', train_data=trn2, preproc=preproc2, metrics=['accuracy'])

Is Multi-Label? False
compiling word ID features...
maxlen is 150
building document-term matrix... this may take a few moments...
rows: 1-7678
computing log-count ratios...
done.


## The Dataset class for binding the audio data and the text data

In [58]:
import math
class MyCustomDataset(ktrain.SequenceDataset):
    def __init__(self, x, y, batch_size=32, shuffle=True):
        # error checks
        err = False
        if type(x) == np.ndarray and len(x.shape) != 2: err = True
        elif type(x) == list:
            for d in x:
                if type(d) != np.ndarray or len(d.shape) != 2:
                    err = True
                    break
        else: err = True
        if err:
            raise ValueError('x must be a 2d numpy array or a list of 2d numpy arrays')
        if type(y) != np.ndarray:
            raise ValueError('y must be a numpy array')
        if type(x) == np.ndarray:
            x = [x]
        # set variables

        super().__init__(batch_size=batch_size)
        self.x, self.y = x, y
        self.indices = np.arange(self.x[0].shape[0])
        self.n_inputs = len(x)
        self.shuffle = shuffle

    # required for instances of tf.keras.utils.Sequence
    def __len__(self):
        """Number of batches in the Sequence.
        """
        return math.ceil(self.x[0].shape[0] / self.batch_size)

    # required for instances of tf.keras.utils.Sequence
    def __getitem__(self, idx):
        """Gets batch at position `idx`.
        """
        # the starting and ending indices of the batch
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = []

        for i in range(self.n_inputs):
            batch_x.append(self.x[i][inds])
        batch_y = np.array(self.y[inds])
        return tuple(batch_x) , batch_y

    # required for instances of ktrain.Dataset
    def nsamples(self):
        return self.x[0].shape[0]

    #required for instances of ktrain.Dataset
    def get_y(self):
        return self.y

    def on_epoch_end(self):
        if self.shuffle:  np.random.shuffle(self.indices)

In [59]:
train_data = MyCustomDataset([num_train] + [trn2[0]], trn2[1])
valid_data = MyCustomDataset([num_valid] + [val2[0]], val2[1])

# Combined model
- the model takes the audio data and the combined train_data, valid_data.
- the purpose of the class is to grid search the best hyperparameters.

In [60]:
class AudioTextGenreClassifier:
    def __init__(self,num_train, valid_train, train_data,num_train_y,valid_data,num_valid_y,class_weight,early_stop,experiment,text_model):
        # num_train data for sequential model input shape
        self.num_train = num_train
        self.valid_train = valid_train
        
        # customed dataset
        self.train_data = train_data
        self.valid_data = valid_data
        self.num_train_y = num_train_y
        self.num_valid_y = num_valid_y
        # text model and data inputs
        self.text_model = text_model

        # class weights and early stopping
        self.class_weight = class_weight
        self.early_stop = early_stop

        # comet experiment
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        model.add(
            Dense(
                self.experiment.get_parameter("first_layer_units"),
                activation='elu',
                input_shape=(self.num_train.shape[1],)
            )
        )
        model.add(Dropout(0.5))
        model.add(BatchNormalization())

        model.add(
            Dense(
                self.experiment.get_parameter("second_layer_units"),
                activation='elu'))

        model.add(Dropout(0.3))
        model.add(BatchNormalization())

        merged_out = keras.layers.concatenate([model.output, self.text_model.output])
        merged_out = keras.layers.Dense(
                        self.experiment.get_parameter("third_layer_units"), 
                        activation='elu',
                        kernel_initializer='HeNormal')(merged_out)

        merged_out = keras.layers.Dense(8, activation='softmax')(merged_out)
        
        combined_model = keras.Model([model.input] + [self.text_model.input], [merged_out])
        combined_model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        return combined_model
    
    def train_model(self):
        # Train the model
        combined_model = self.build_model()
        model = ktrain.get_learner(
            combined_model,
            train_data=self.train_data,
            val_data=self.valid_data,
            batch_size=self.experiment.get_parameter("batch_size")
        )
        model.fit_onecycle(
            lr=0.001,
            epochs=self.experiment.get_parameter("epochs"),
            class_weight=self.class_weight,
        )

        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        score = model.evaluate(test_data=self.valid_data)
        # LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):
            # self.experiment.log_parameters("epochs", 10)

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

## Grid search
- using the comet_ml library, we can track the hyperparameters and metrics and all the metrics are on the comet_ml dashboard.

In [61]:
audio_model = AudioTextGenreClassifier(
    num_train,
    num_valid,
    train_data,
    num_train_y,
    valid_data,
    num_valid_y,
    class_weight,
    early_stop,
    experiment,
    model2,
)
# mu is the mean number of units, sigma is the standard deviation
audio_model.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {"type": "discrete", "values": [800,850,900,950,1000,1050,1100,1150,1200,1250]},
        "second_layer_units": {"type": "discrete", "values": [50,60,75,100,150,200,225,250,275,300]},
        "third_layer_units": {"type": "discrete", "values": [16,32]},
        "batch_size": {"type": "discrete", "values": [4,8,16,32]},
        "epochs": {"type": "discrete", "values": [4,5,6]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=b7ab9e4374024f4ab06872d8faa3a9d6
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 2400, 'endTime': None, 'id': 'b7ab9e4374024f4ab06872d8faa3a9d6', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [4, 8, 16, 32]}, 'epochs': {'type': 'discrete', 'values': [4, 5, 6]}, 'first_layer_units': {'type': 'discrete', 'values': [800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250]}, 'second_layer_units': {'type': 'discrete', 'values': [50, 60, 75, 100, 150, 200, 225, 250, 275, 300]}, 'third_layer_units': {'type': 'discrete', 'values': [16, 32]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 26907189876, 'state': {'mode': None, 'seed': None, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequenc



begin training using onecycle policy with max lr of 0.001...
Epoch 1/6
240/240 [==============================] - 2s 7ms/step - loss: 1.8858 - accuracy: 0.3108 - val_loss: 1.4742 - val_accuracy: 0.4467
Epoch 2/6
240/240 [==============================] - 2s 8ms/step - loss: 1.5407 - accuracy: 0.4144 - val_loss: 1.4083 - val_accuracy: 0.5089
Epoch 3/6
240/240 [==============================] - 1s 5ms/step - loss: 1.4034 - accuracy: 0.4743 - val_loss: 1.3603 - val_accuracy: 0.5400
Epoch 4/6
240/240 [==============================] - 1s 5ms/step - loss: 1.3144 - accuracy: 0.5090 - val_loss: 1.3426 - val_accuracy: 0.5511
Epoch 5/6
240/240 [==============================] - 1s 4ms/step - loss: 1.2287 - accuracy: 0.5449 - val_loss: 1.2655 - val_accuracy: 0.5778
Epoch 6/6
240/240 [==============================] - 1s 4ms/step - loss: 1.1620 - accuracy: 0.5698 - val_loss: 1.2201 - val_accuracy: 0.6000


begin training using onecycle policy with max lr of 0.001...
Epoch 1/6
240/240 [=========

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/706226589c134020a7e946669c8e2088
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [12]                : (0.3107580244541168, 0.5767126679420471)
COMET INFO:     batch_accuracy [288]         : (0.09375, 0.65625)
COMET INFO:     batch_loss [288]             : (1.0686818361282349, 3.1361868381500244)
COMET INFO:     epoch_duration [12]          : (1.0930000000007567, 2.312999999994645)
COMET INFO:     loss [12]                    : (1.1220974922180176, 1.8857786655426025)
COMET INFO:     val_accuracy [12]            : (0.4266666769981384, 0.6155555844306946)
COMET INFO:     val_loss [12]                : (1.1455631256103516, 1.5079718828201294)
COMET INFO:     validate_batch_accu



begin training using onecycle policy with max lr of 0.001...
Epoch 1/5
480/480 [==============================] - 3s 5ms/step - loss: 1.8190 - accuracy: 0.3253 - val_loss: 1.3871 - val_accuracy: 0.5422
Epoch 2/5
480/480 [==============================] - 2s 5ms/step - loss: 1.5195 - accuracy: 0.4264 - val_loss: 1.2648 - val_accuracy: 0.5711
Epoch 3/5
480/480 [==============================] - 2s 5ms/step - loss: 1.3655 - accuracy: 0.4861 - val_loss: 1.2524 - val_accuracy: 0.5756
Epoch 4/5
480/480 [==============================] - 2s 5ms/step - loss: 1.2525 - accuracy: 0.5376 - val_loss: 1.1755 - val_accuracy: 0.6067
Epoch 5/5
480/480 [==============================] - 3s 7ms/step - loss: 1.1740 - accuracy: 0.5623 - val_loss: 1.1152 - val_accuracy: 0.6133


begin training using onecycle policy with max lr of 0.001...
Epoch 1/5
480/480 [==============================] - 4s 6ms/step - loss: 1.8259 - accuracy: 0.3076 - val_loss: 1.4571 - val_accuracy: 0.4644
Epoch 2/5
480/480 [=========

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/7ae4c730cee14126ba2b28829a857949
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.3076322078704834, 0.5636884570121765)
COMET INFO:     batch_accuracy [480]         : (0.0, 0.75)
COMET INFO:     batch_loss [480]             : (0.793403148651123, 3.1721742153167725)
COMET INFO:     epoch_duration [10]          : (2.264999999999418, 3.6089999999967404)
COMET INFO:     loss [10]                    : (1.1740434169769287, 1.825923204421997)
COMET INFO:     val_accuracy [10]            : (0.46444445848464966, 0.6311110854148865)
COMET INFO:     val_loss [10]                : (1.115221381187439, 1.4571149349212646)
COMET INFO:     validate_batch_accuracy [20]

...

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/79bc66672d5d4fc79e560d1ed2a615b1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [8]                 : (0.34032300114631653, 0.5941651463508606)
COMET INFO:     batch_accuracy [768]         : (0.10227272659540176, 0.625)
COMET INFO:     batch_loss [768]             : (0.7683548331260681, 3.127572774887085)
COMET INFO:     epoch_duration [8]           : (4.265999999995984, 7.9060000000026776)
COMET INFO:     loss [8]                     : (1.0489561557769775, 1.879279375076294)
COMET INFO:     val_accuracy [8]             : (0.42888888716697693, 0.648888885974884)
COMET INFO:     val_loss [8]                 : (1.057554006576538, 1.577625036239624)
COMET INFO:     validate_bat



begin training using onecycle policy with max lr of 0.001...
Epoch 1/5
960/960 [==============================] - 5s 5ms/step - loss: 1.8311 - accuracy: 0.3248 - val_loss: 1.4273 - val_accuracy: 0.4756
Epoch 2/5
960/960 [==============================] - 4s 5ms/step - loss: 1.4939 - accuracy: 0.4398 - val_loss: 1.2315 - val_accuracy: 0.5778
Epoch 3/5
960/960 [==============================] - 5s 5ms/step - loss: 1.2794 - accuracy: 0.5223 - val_loss: 1.1078 - val_accuracy: 0.6178
Epoch 4/5
960/960 [==============================] - 4s 5ms/step - loss: 1.1111 - accuracy: 0.5768 - val_loss: 1.1382 - val_accuracy: 0.6044
Epoch 5/5
960/960 [==============================] - 6s 7ms/step - loss: 1.0188 - accuracy: 0.6149 - val_loss: 1.1374 - val_accuracy: 0.6244


begin training using onecycle policy with max lr of 0.001...
Epoch 1/5
960/960 [==============================] - 7s 6ms/step - loss: 1.8172 - accuracy: 0.3403 - val_loss: 1.4692 - val_accuracy: 0.4356
Epoch 2/5
960/960 [=========

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/ef5c82dad70e44c8974e12740d743e03
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.32482418417930603, 0.6148736476898193)
COMET INFO:     batch_accuracy [960]         : (0.125, 0.875)
COMET INFO:     batch_loss [960]             : (0.7887321710586548, 3.0660574436187744)
COMET INFO:     epoch_duration [10]          : (4.328000000001339, 6.89100000000326)
COMET INFO:     loss [10]                    : (1.0187840461730957, 1.8311457633972168)
COMET INFO:     val_accuracy [10]            : (0.4355555474758148, 0.6244444251060486)
COMET INFO:     val_loss [10]                : (1.1027315855026245, 1.4691952466964722)
COMET INFO:     validate_batch_accuracy 



begin training using onecycle policy with max lr of 0.001...
Epoch 1/4
1920/1920 [==============================] - 10s 5ms/step - loss: 1.8877 - accuracy: 0.3140 - val_loss: 1.3648 - val_accuracy: 0.5267
Epoch 2/4
1920/1920 [==============================] - 12s 6ms/step - loss: 1.4174 - accuracy: 0.4573 - val_loss: 1.2471 - val_accuracy: 0.5356
Epoch 3/4
1920/1920 [==============================] - 12s 6ms/step - loss: 1.2454 - accuracy: 0.5029 - val_loss: 1.2302 - val_accuracy: 0.5711
Epoch 4/4
1920/1920 [==============================] - 9s 5ms/step - loss: 1.1593 - accuracy: 0.5258 - val_loss: 1.1949 - val_accuracy: 0.5889


begin training using onecycle policy with max lr of 0.001...
Epoch 1/4
1920/1920 [==============================] - 13s 6ms/step - loss: 1.8954 - accuracy: 0.3139 - val_loss: 1.4254 - val_accuracy: 0.5133
Epoch 2/4
1920/1920 [==============================] - 11s 6ms/step - loss: 1.4656 - accuracy: 0.4406 - val_loss: 1.2725 - val_accuracy: 0.5533
Epoch 3/4
1

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/d4d0eb68f89448a29a6db833c83be8c9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [8]                 : (0.31388381123542786, 0.5311278700828552)
COMET INFO:     batch_accuracy [1536]        : (0.0, 1.0)
COMET INFO:     batch_loss [1536]            : (0.9748262166976929, 3.861923933029175)
COMET INFO:     epoch_duration [8]           : (8.81300000000192, 12.5)
COMET INFO:     loss [8]                     : (1.147010087966919, 1.8953789472579956)
COMET INFO:     val_accuracy [8]             : (0.5133333206176758, 0.602222204208374)
COMET INFO:     val_loss [8]                 : (1.194944977760315, 1.4253896474838257)
COMET INFO:     validate_batch_accuracy [16] : (0.5, 0.625)
C

# Build and save the best model for prediction

In [70]:
model = Sequential()
model.add(
    Dense(
        1100,
        activation='elu',
        input_shape=(num_train.shape[1],)
    )
)
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(
    Dense(
        75,
        activation='elu'))

model.add(Dropout(0.3))
model.add(BatchNormalization())

merged_out = keras.layers.concatenate([model.output, model2.output])
merged_out = keras.layers.Dense(
                64, 
                activation='elu',
                kernel_initializer='HeNormal')(merged_out)

merged_out = keras.layers.Dense(8, activation='softmax')(merged_out)

combined_model = keras.Model([model.input] + [model2.input], [merged_out])
combined_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [71]:
learner = ktrain.get_learner(
            combined_model,
            train_data=train_data,
            val_data=valid_data,
            batch_size=32
        )
learner.fit_onecycle(
    lr=0.001,
    epochs=5,
    class_weight=class_weight,
)



begin training using onecycle policy with max lr of 0.001...
Epoch 1/5
240/240 [==============================] - 2s 6ms/step - loss: 1.8602 - accuracy: 0.3510 - val_loss: 1.4409 - val_accuracy: 0.4844
Epoch 2/5
240/240 [==============================] - 1s 6ms/step - loss: 1.3889 - accuracy: 0.4794 - val_loss: 1.2850 - val_accuracy: 0.5378
Epoch 3/5
240/240 [==============================] - 1s 5ms/step - loss: 1.1350 - accuracy: 0.5800 - val_loss: 1.1731 - val_accuracy: 0.6156
Epoch 4/5
240/240 [==============================] - 1s 5ms/step - loss: 0.9859 - accuracy: 0.6269 - val_loss: 1.1435 - val_accuracy: 0.5911
Epoch 5/5
240/240 [==============================] - 1s 5ms/step - loss: 0.9077 - accuracy: 0.6470 - val_loss: 1.1214 - val_accuracy: 0.6178


In [72]:
predictor = ktrain.get_predictor(model=learner.model, preproc=preproc2)

In [73]:
predictor.save('audio_text_nbsvm_model')